### 摘要

This paper *describes*（描述） a visual object *detection*（检测） framework that *is capable of*（能够） processing images *extremely*（非常） *rapidly*（迅速地） while achieving high detection rates.<bt>
本文描述了一种视觉对象检测框架，它可以非常快速地处理图像，同时实现高检测率。

There are three key contributions.<br>
他有三个关键的贡献。

The first is the *introduction*（引入） of a new image *representation*（表示） called the “Integral Image” which allows the features used by our *detector*（探测器） to be computed very quickly.<br>
第一个是引入一个新的图像表示--“积分图像”, 可以让我们的探测器使用的特征进行非常快的计算。


The second is a learning algorithm based on AdaBoost, which selects a small number of *critical*（要紧，关键） visual features and *yields*（生产） *extremely*（非常） *efficient*（有效） classifiers.[4]<br>
第二种是基于AdaBoost的学习算法，它选择一个小的关键视觉特征的数量和产生极其有效的分类器

The third contribution is a method for *combining*（化和的，结合的） classifiers in a “cascade” which allows background *regions*（区域） of the image to be quickly *discarded*（丢弃） while spending more computation on *promising*（有希望的） object-like *regions.*（区域）<br>
第三个贡献是在“级联”中分类器组合的方法允许图像的背景区域被快速丢弃，同时花费更多的计算在有前途的对象状区域。

A set of *experiments*（实验） in the domain of face detection are presented. The system yields face detection performance（性能） comparable to the best previous systems [16, 11, 14, 10, 1]. Implemented on a conventional desktop, face detection proceeds at 15 frames per second.<br>
介绍了一系列面部检测领域的实验。 该系统的人脸检测性能可与先前最好的系统相媲美[16,11,14,10,1]。 在传统桌面上执行，脸部检测以每秒15帧的速度进行。


This work may not be copied or reproduced in whole or in part for any commercial purpose. Permission to copy in whole or in part without payment of fee is granted for nonprofit educational and research purposes provided that all such whole or partial copies include the following: a notice that such copying is by permission of the Cambridge Research Laboratory of Compaq Computer Corporation in Cambridge, Massachusetts; an acknowledgment of the authors and individual contributors to the work; and all applicable portions of the copyright notice. Copying, reproducing, or republishing for any other purpose shall require a license with payment of fee to the Cambridge Research Laboratory. All rights reserved.


[4] Yoav Freund and Robert E. Schapire. A decision-theoretic generalization of online learning and an application to boosting. InComputational Learning Theory:
Eurocolt ’95, pages 23–37. Springer-Verlag, 1995.

### 1 Introduction（介绍）

This paper brings together new algorithms and insights to construct a framework for robust and extremely rapid object detection.<br>
本文汇集了新的算法和见解，构建了一个强大且极其快速的对象检测框架。

This framework is demonstrated on, and in part motivated by, the task of face detection. <br>
这个框架在人脸检测的任务上进行了演示，部分是由它的动机决定的。

Toward this end we have constructed a frontal face detection system which achieves detection and false positive rates which are equivalent to the best published results [16, 11, 14, 10, 1].
为此，我们构建了一个正面人脸检测系统，该系统可以实现检测和误报率，这相当于公布的最佳结果[16,11,14,10,1]

This face detection system is most clearly distinguished from previous approaches in its ability to detect faces extremely rapidly. Operating on 384 by 288 pixel images, faces are detected at 15 frames per second on a conventional 700 MHz Intel Pentium III. <br>
这种人脸检测系统与以前的方法在能够非常迅速地检测人脸方面的区别很明显。使用384 x 288像素的图像进行操作时，在常规700 MHz Intel Pentium III上以15帧/秒的速度检测面部。

In other face detection systems, auxiliary information, such as image differences in video sequences, or pixel color in color images, have been used to achieve high frame rates. Our system achieves high frame rates working only with the information present in a single grey scale image. These alternative sources of information can also be integrated with our system to achieve even higher frame rates.<br>
在其他人脸检测系统中，辅助信息（例如视频序列中的图像差异或彩色图像中的像素色彩）已被用于实现高帧率。我们的系统只能处理单个灰度图像中的信息，实现了高帧率。这些替代信息来源也可以与我们的系统集成以实现更高的帧速率。

There are three main contributions of our object detection framework. We will introduce each of these ideas briefly below and then describe them in detail in subsequent sections.<br>
我们的对象检测框架有三个主要贡献。我们将在下面简要介绍这些想法，然后在随后的章节中详细介绍它们。

The first contribution of this paper is a new image representation called anintegral image that allows for very fast feature evaluation. Motivated in part by the work of Papageorgiou et al. our detection system does not work directly with image intensities [8]. Like these authors we use a set of features which are reminiscent of Haar Basis functions (though we will also use related filters which are more complex than Haar filters). In order to compute these features very rapidly at many scales we introduce the integral image representation for images (the integral image is very similar to the summed area table used in computer graphics [2] for texture mapping). The integral image can be computed from an image using a few operations per pixel. Once computed, any one of these Harr-like features can be computed at any scale or location in constanttime.<br>
本文的第一个贡献是一种称为整体图像的新图像表示，它允许进行非常快速的特征评估。部分受到Papageorgiou等人工作的启发。我们的检测系统不能直接处理图像强度[8]。像这些作者一样，我们使用了一组让人联想到Haar Basis函数的特性（尽管我们也会使用比Haar滤波器更复杂的相关滤波器）。为了在许多尺度上非常快速地计算这些特征，我们引入了图像的积分图像表示（积分图像非常类似于用于纹理映射的计算机图形[2]中使用的求和区域表）。积分图像可以使用每像素几个操作从图像计算。一旦计算出来，这些类似Harr的特征中的任何一个都可以在任何比例或位置的恒定时间内进行计算。

The second contribution of this paper is a method for constructing a classifier by selecting a small number of important features using AdaBoost [4]. Within any image sub-window the total number of Harr-like features is very large, far larger than the number of pixels. In order to ensure fast classification, the learning process must exclude a large majority of the available features, and focus on a small set of critical features. Motivated by the work of Tieu and Viola, feature selection is achieved through a simple modification of the AdaBoost procedure: the weak learner is constrained so that each weak classifier returned can depend on only a single feature [17]. As a result each stage of the boosting process, which selects a new weak classifier, can be viewed as a feature selection process. AdaBoost provides an effective learning algorithm and strong bounds on generalization performance [12, 7, 8].<br>
本文的第二个贡献是通过使用AdaBoost [4]选择少量重要特征来构建分类器的方法。在任何图像子窗口内，Harr-like特征的总数量非常大，远大于像素数量。为了确保快速分类，学习过程必须排除大部分可用功能，并关注一小组关键功能。受Tieu和Viola工作的启发，特征选择是通过对AdaBoost过程的简单修改实现的：弱学习者受到约束，使得每个弱分类器只能依赖一个特征[17]。因此，选择新的弱分类器的助推过程的每个阶段都可以被视为特征选择过程。 AdaBoost提供了一种有效的学习算法和泛化性能的强大界限[12,7,8]。

The third major contribution of this paper is a method for combining successively more complex classifiers in a cascade structure which dramatically increases the speed of the detector by focussing attention on promising regions of the image. The notion behind focus of attention approaches is that it is often possible to rapidly determine where in an image an object might occur [18, 6, 1]. More complex processing is reserved only for these promising regions. The key measure of such an approach is the “false negative” rate of the attentional process. It must be the case that all, or almost all, object instances are selected by the attentional filter.<br>
本文的第三个主要贡献是相继组合的方法 级联结构中更复杂的分类器可显着提高速度 通过将注意力集中在图像的有前途的区域上。这个概念 后面关注焦点的方法是通常可以迅速确定 在图像中可能出现对象[18,6,1]。更复杂的处理仅保留给这些有前途的地区。这种方法的关键措施是注意过程的“假否定”率。一切都是这样，或者差不多 所有对象实例都由注意过滤器选择。

We will describe a process for training an extremely simple and efficient classifier which can be used as a “supervised” focus of attention operator. The term supervised refers to the fact that the attentional operator is trained to detect examples of a particular class. In the domain of face detection it is possible to achieve fewer than 1% false negatives and 40% false positives using a classifier which can be evaluated in 20 simple operations (approximately 60 microprocessor instructions). The effect of this filter is to reduce by over one half the number of locations where the final detector must be evaluated.<br>
我们将描述一个训练一个非常简单和有效的分类器的过程，该分类器可以作为一个“监督”的关注操作者焦点。术语监督指的是注意力操作者被训练来检测特定类别的例子。在人脸检测领域，使用可在20个简单操作（大约60个微处理器指令）中评估的分类器，可以实现少于1％的假阴性和40％的假阳性。该滤波器的作用是将最终检测器必须评估的位置数量减少一半以上。

Those sub-windows which are not rejected by the initial classifier are processed by a sequence of classifiers, each slightly more complex than the last. If any classifier rejects the sub-window, no further processing is performed. The structure of the cascaded detection process is essentially that of a degenerate decision tree, and as such is related to the work of Amit and Geman [1].<br>
那些未被初始分类器拒绝的子窗口由一系列分类器处理，每个分类器比最后一个稍微复杂。如果任何分类器拒绝子窗口，则不执行进一步的处理。级联检测过程的结构基本上是退化决策树的结构，因此与Amit和Geman的工作有关

The complete face detection cascade has 32 classifiers, which total over 80,000 operations. Nevertheless the cascade structure results in extremely rapid average detection times. On a difficult dataset, containing 507 faces and 75 million sub-windows, faces are detected using an average of 270 microprocessor instructions per sub-window. In comparison, this system is about 15 times faster than an implementation of the detection system constructed by Rowley et al.<br>
完整的人脸检测级联有32个分类器，总计超过80,000次操作。尽管如此，级联结构的平均检测时间非常快。在一个包含507个面和7500万个子窗口的困难数据集中，每个子窗口使用平均270个微处理器指令检测面。相比之下，这个系统比由Rowley等人构建的检测系统的实现快大约15倍。

An extremely fast face detector will have broad practical applications. These include user interfaces, image databases, and teleconferencing. This increase in speed will enable real-time face detection applications on systems where they were previously infeasible. In applications where rapid frame-rates are not necessary, our system will allow for significant additional post-processing and analysis. In addition our system can be implemented on a wide range of small low power devices, including handhelds and embedded processors. In our lab we have implemented this face detector on the Compaq iPaq handheld and have achieved detection at two frames per second (this device has a low power 200 mipsStrong Armprocessor which lacks floating point hardware).<br>
一个非常快的人脸检测器将有广泛的实际应用。这些包括用户界面，图像数据库和电话会议。速度的提高将使以前不可行的系统上的实时人脸检测应用成为可能。在不需要快速帧率的应用中，我们的系统将允许进行大量额外的后处理和分析。此外，我们的系统可以在各种小型低功耗设备上实施，包括手持设备和嵌入式处理器。在我们的实验室中，我们在Compaq iPaq手持设备上实现了这种面部检测器，并且已经实现了每秒两帧的检测（该设备具有低功率200 mips强大的Armprocessor，它缺少浮点硬件）。

#### 1.1 Overview 

The remaining sections of the paper will discuss the implementation of the detector,related theory, and experiments. Section 2 will detail the form of the features as well as a new scheme for computing them rapidly. Section 3 will discuss the method in which these features are combined to form a classifier. The machine learning method used, a variant of AdaBoost, also acts as a feature selection mechanism. While the classifiers that are constructed in this way have good computational and classification performance, they are far too slow for a real-time classifier. Section 4 will describe a method for constructing a cascade of classifiers which together yield an extremely reliable and efficient object detector. Section 5 will describe a number of experimental results, including a detailed description of our experimental methodology. Finally Section 6 contains a discussion of this system and its relationship to related systems.<br>
本文的其余部分将讨论检测器的实施情况，相关理论和实验。第2部分将详细介绍这些功能的形式以及快速计算它们的新方案。第3节将讨论将这些特征组合起来形成分类器的方法。所使用的机器学习方法（AdaBoost的一种变体）也充当特征选择机制。虽然以这种方式构建的分类器具有良好的计算和分类性能，但它们对于实时分类器来说太慢了。第4节将介绍一种构建级联分类器的方法，它们一起产生极其可靠和高效的对象检测器。第5节将介绍一些实验结果，包括我们实验方法的详细描述。最后，第6节讨论了这个系统及其与相关系统的关系。

![Figure 1](img/1-1.png)

### 2 Features

Our object detection procedure classifies images based on the value of simple features. There are many motivations for using features rather than the pixels directly. The most common reason is that features can act to encode ad-hoc domain knowledge that is difficult to learn using a finite quantity of training data. For this system there is also a second critical motivation for features: the feature-based system operates much faster than a pixel-based system.<br>
我们的目标检测程序根据简单特征的值对图像进行分类。使用特征而不是直接使用像素有很多动机。最常见的原因是特征可以用于对使用有限数量的训练数据难以学习的特定领域知识进行编码。对于这个系统，还有第二个关键功能的动机：基于特征的系统的运行速度比基于像素的系统快得多。

The simple features used are reminiscent of Haar basis functions which have been used by Papageorgiou et al. [8]. More specifically, we use three kinds of features. The value of atwo-rectangle featureis the difference between the sum of the pixels within two rectangular regions. The regions have the same size and shape and are horizontally or vertically adjacent (see Figure 1). Athree-rectangle featurecomputes the sum within two outside rectangles subtracted from the sum in a center rectangle. Finally a fourrectangle featurecomputes the difference between diagonal pairs of rectangles.<br>
使用的简单特征让人联想到Papageorgiou等人使用的哈尔基函数。 [8]。更具体地说，我们使用三种功能。 atwo-rectangle特征的值是两个矩形区域内像素之和的差值。这些区域具有相同的大小和形状，并且水平或垂直相邻（见图1）。 Athree矩形特征计算两个外部矩形内的和，并从中心矩形的总和中减去。最后，四矩形特征计算对角矩形对之间的差异。

Given that the base resolution of the detector is 24x24, the exhaustive set of rectangle features is quite large, 45,396 . Note that unlike the Haar basis, the set of rectangle features is overcomplete.<br>
假设检测器的基本分辨率是24x24，那么矩形特征的详尽集合是非常大的，45,396。请注意，与哈尔基不同，矩形特征集过度完整。

![Figure 2](img/1-2.png)


#### 2.1 Integral Image

Rectangle features can be computed very rapidly using an intermediate representation for the image which we call the integral image. The integral image at location x; y contains the sum of the pixels above and to the left ofx; y, inclusive:<br>
使用图像的中间表示可以非常快速地计算矩形特征，我们称之为积分图像。位置x处的积分图像; y包含x上方和左边的像素之和; y，包括：

$$ ii(x, y) = \sum_{x'<=x, y'<=y}i(x', y') $$

where ii(x; y)is the integral image and i(x; y) is the original image (see Figure 2).Using the following pair of recurrences:<br>
其中ii（x; y）是积分图像，i（x; y）是原始图像（参见图2）。使用以下一对重复：

$$ s(x, y) = s(x, y - 1) + i(x, y) $$
$$ ii(x, y) = ii(x-1, y) + s(x, y) $$

(where \\(s(x, y)\\) is the cumulative row sum, \\(s(x, -1) = 0\\), and \\(ii(-1, y) = 0\\)) the integral image can be computed in one pass over the original image.<br>
其中\\（s（x，y）\\）是累积行和，\\（s（x，-1）= 0 \\），并且\\（ii（-1，y）= 0 \ \））可以在原始图像上一次通过计算积分图像

Using the integral image any rectangular sum can be computed in four array references (see Figure 3). Clearly the difference between two rectangular sums can be computed in eight references. Since the two-rectangle features defined above involve adjacent rectangular sums they can be computed in six array references, eight in the case of the three-rectangle features, and nine for four-rectangle features.<br>
使用积分图像可以在四个数组参考中计算任意矩形和（参见图3）。很显然，两个矩形和之间的差异可以用八个参考文件来计算。由于上面定义的两个矩形要素涉及相邻的矩形和，因此可以在六个数组参考中计算它们，在三个矩形特征的情况下为八个，而在四个矩形特征中为九个。

![Figure 3](img/1-3.png)


One alternative motivation for the integral image comes from the “boxlets” work of Simard, et al. [15]. The authors point out that in the case of linear operations (e.g.f\*g), any invertible linear operation can be applied toforgif its inverse is applied to the result. For example in the case of convolution, if the derivative operator is applied both to the image and the kernel the result must then be double integrated:<br>
整体图像的一个替代动机来自Simard等人的“boxlets”作品。作者指出，在线性运算的情况下（例如f * g），任何可逆的线性运算都可以应用于其反演结果。例如，在卷积的情况下，如果导数运算符同时应用于图像和内核，则必须对结果进行双重积分：

$$ f * g = \int\int(f' * g') $$

The authors go on to show that convolution can be significantly accelerated if the derivatives offandgare sparse (or can be made so). A similar insight is that an invertible linear operation can be applied tofif its inverse is applied to g:<br>
作者继续证明，如果衍生品稀少（或可以这样做），卷积可以显着加速。类似的见解是，可逆线性运算可应用于其逆应用于g：

$$ (f'') * (\int\int g) = f * g $$

Viewed in this framework computation of the rectangle sum can be expressed as adot product,ir, whereiis the image andris the box car image (with value 1 within<br>
在这个框架中，矩形和的计算可以表示为adot乘积，i r，其中ii是图像，并且是箱形车图像（其值为1

$$ i · r = (\int\int i) · r'' $$

The integral image is in fact the double integral of the image (first along rows and then along columns). The second derivative of the rectangle (first in row and then in column) yields four delta functions at the corners of the rectangle. Evaluation of the second dot product is accomplished with four array accesses<br>
积分图像实际上是图像的双重积分（首先沿着行，然后沿着列）。矩形的二阶导数（首先在行中，然后在列中）在矩形的角上产生四个三角函数。第二点产品的评估是通过四次阵列访问完成的

#### 2.2 Feature Discussion(专题讨论)

Rectangle features are somewhat primitive when compared with alternatives such as steerable filters [3, 5]. Steerable filters, and their relatives, are excellent for the detailed analysis of boundaries, image compression, and texture analysis. In contrast rectangle features, while sensitive to the presence of edges, bars, and other simple image structure, are quite coarse. Unlike steerable filters the only orientations available are vertical and horizontal. It appears as though the set of rectangle features do however provide a rich image representation which supports effective learning. The extreme computational efficiency of rectangle features provides ample compensation for their limited flexibility.<br>
与可选滤波器等替代方法相比，矩形特征有点原始[3，5]。可操纵过滤器及其亲属非常适合详细分析边界，图像压缩和纹理分析。相比之下，矩形特征虽然对边缘，条纹和其他简单图像结构的存在很敏感，但却非常粗糙。与可调滤镜不同，可用的唯一方向是垂直和水平方向。看起来好像这组矩形特征确实提供了支持有效学习的丰富图像表示。矩形特征的极端计算效率为其有限的灵活性提供了充足的补偿。

In order to appreciate the computational advantage of the integral image technique,consider a more conventional approach in which a pyramid of images is computed.Like most object detection systems, our detector scans the input at many scales; starting at the base scale in which objects are detected at a size of 24x24 pixels, the image is scanned at 11 scales each a factor of 1.25 larger than the last. The conventional approach is to compute a pyramid of 11 images, each 1.25 times smaller than the previous image. A fixed scale detector is then scanned across each of these images. Computation of the pyramid, while straightforward, requires significant time. Implemented on conventional hardware it is extremely difficult to compute a pyramid at 15 frames per second.<br>
为了理解积分图像技术的计算优势，考虑一种更常规的方法，其中计算金字塔图像。与大多数物体检测系统一样，我们的检测器以多种比例扫描输入;从以24x24像素大小检测物体的基本比例开始，以11比例扫描图像，每个比例比上一个大1.25倍。常规方法是计算11幅图像的金字塔，每幅图像比前一幅图像小1.25倍。然后在这些图像中的每一个上扫描固定比例的检测器。计算金字塔虽然简单，但需要大量时间。在传统硬件上实现，以每秒15帧计算金字塔是非常困难的。


In contrast we have defined a meaningful set of features, which have the property that a single feature can be evaluated at any scale and location in a few operations. We will show in Section 4 that effective face detectors can be constructed with as little as two rectangle features. Given the computational efficiency of these features, the face detection process can be completed for an entire image at every scale at 15 frames per second, less time than is required to evaluate the 11 level image pyramid alone. Any procedure which requires a pyramid of this type will necessarily run slower than our detector.<br>
相比之下，我们定义了一组有意义的特征，这些特征具有可以在一些操作中的任何尺度和位置评估单个特征的特性。我们将在第4节中展示有效的人脸检测器可以构造成只有两个矩形特征。考虑到这些特征的计算效率，可以以每秒15帧的速度在每个比例的整个图像上完成人脸检测过程，比仅评估11级图像金字塔所需的时间少。任何需要这种金字塔的程序必然会比我们的探测器运行得慢

### 3 Learning Classification Functions(学习分类函数)

Given a feature set and a training set of positive and negative images, any number of machine learning approaches could be used to learn a classification function. Sung and Poggio use a mixture of Gaussian model [16]. Rowley, Baluja, and Kanade use a small set of simple image features and a neural network [11]. Osuna, et al. used a support vector machine [7]. More recently Roth et al. have proposed a new and unusual image representation and have used the Winnow learning procedure [10].<br>
给定一个特征集和正负图像的训练集，可以使用任意数量的机器学习方法来学习分类函数。 Sung和Poggio使用高斯模型的混合[16]。 Rowley，Baluja和Kanade使用一组简单的图像特征和一个神经网络[11]。 Osuna等人使用支持向量机[7]。最近Roth等人已经提出了一个新的和不寻常的图像表示，并使用了Winnow学习过程[10]。

Recall that there are 45,396 rectangle features associated with each image subwindow, a number far larger than the number of pixels. Even though each feature can be computed very efficiently, computing the complete set is prohibitively expensive. Our hypothesis, which is borne out by experiment, is that a very small number of these features can be combined to form an effective classifier. The main challenge is to find these features.<br>
回想一下，每个图像子窗口都有45,396个矩形特征，这个数字远大于像素数量。即使每个功能都可以非常有效地计算，但计算整套设备的代价非常昂贵。我们的假设是通过实验证明的，只有极少数这些特征可以结合起来形成一个有效的分类器。主要挑战是找到这些功能。

In our system a variant of AdaBoost is used both to select the features and to train the classifier. In its original form, the AdaBoost learning algorithm is used to boost the classification performance of a simple learning algorithm (e.g., it might be used to boost the performance of a simple perceptron). It does this by combining a collection of weak classification functions to form a stronger classifier. In the language of boosting the simple learning algorithm is called a weak learner. So, for example the perceptron learning algorithm searches over the set of possible perceptrons and returns the perceptron with the lowest classification error. The learner is called weak because we do not expect even the best classification function to classify the training data well (i.e. for a given problem the best perceptron may only classify the training data correctly 51% of the time). In order for the weak learner to be boosted, it is called upon to solve a sequence of learning problems. After the first round of learning, the examples are re-weighted in order to emphasize those which were incorrectly classified by the previous weak classifier. The final strong classifier takes the form of a perceptron, a weighted combination of weak classifiers followed by a threshold.<br>
在我们的系统中，使用AdaBoost的变体来选择特征并训练分类器。在其原始形式中，AdaBoost学习算法用于提高简单学习算法的分类性能（例如，它可以用于提高简单感知器的性能）。它通过组合一组弱分类函数来形成更强的分类器。在提高简单学习算法的语言中被称为弱学习者。因此，例如感知器学习算法搜索可能的感知器集合并返回具有最低分类误差的感知器。学习者被称为弱，因为我们并不期望即使是最好的分类函数也能很好地对训练数据进行分类（即对于给定问题，最好的感知器只能在51％的时间内正确分类训练数据）。为了加强弱势学习者，它被要求解决一系列学习问题。在第一轮学习之后，对这些例子进行重新加权，以强调那些被前一个弱分类器错误分类的例子。最终的强分类器采用感知器的形式，弱分类器的加权组合随后是阈值。

The formal guarantees provided by the AdaBoost learning procedure are quite strong. Freund and Schapire proved that the training error of the strong classifier approaches zero exponentially in the number of rounds. More importantly a number of results were later proved about generalization performance [13]. The key insight is that generalization performance is related to the margin of the examples, and that AdaBoost achieves large margins rapidly.<br>
AdaBoost学习程序提供的正式保证非常强大。 Freund和Schapire证明了强分类器的训练误差在轮次数量上呈指数级增长。更重要的是后来证明了泛化性能的一些结果[13]。关键的见解是泛化性能与示例的差距有关，并且AdaBoost能够快速实现大幅度的利润

The conventional AdaBoost procedure can be easily interpreted as a greedy feature selection process. Consider the general problem of boosting, in which a large set of classification functions are combined using a weighted majority vote. The challenge is to associate a large weight with each good classification function and a smaller weight with poor functions. AdaBoost is an aggressive mechanism for selecting a small set of good classification functions which nevertheless have significant variety. Drawing an analogy between weak classifiers and features, AdaBoost is an effective procedure for searching out a small number of good “features” which nevertheless have significant variety.<br>
传统的AdaBoost过程可以很容易地解释为一个贪婪的特征选择过程。考虑助推的一般问题，其中使用加权多数表决将大量分类函数组合在一起。面临的挑战是将一个较大的权重与每个良好的分类功能和一个功能较差的较小权重相关联。 AdaBoost是一种积极的机制，用于选择一小组好的分类函数，但它们有很大的不同。借助弱分类器和特征之间的类比，AdaBoost是一种搜索少量优秀“特征”的有效方法，但它们有很大的不同。

One practical method for completing this analogy is to restrict the weak learner to the set of classification functions each of which depend on a single feature. In support of this goal, the weak learning algorithm is designed to select the single rectangle feature which best separates the positive and negative examples (this is similar to the approach of [17] in the domain of image database retrieval). For each feature, the weak learner determines the optimal threshold classification function, such that the minimum number of examples are misclassified. A weak classifier \\((h_j(x))\\) thus consists of a feature \\((f_j)\\) a threshold (\\(\Theta_j\\)) and a parity (\ (p_j\\)) indicating the direction of the inequality sign:<br>
完成这种类比的一种实用方法是将弱学习者限制为依赖于单个特征的一组分类函数。为了支持这一目标，弱学习算法被设计为选择最能区分正面和负面示例的单个矩形特征（这与图像数据库检索领域中的[17]的方法类似）。对于每个特征，弱学习者确定最优阈值分类函数，使得最小数量的例子被错误分类。一个弱分类器\\（（h_j（x））\\）因此包括一个特征\\（（f_j）\\）阈值（\\（\ Theta_j \\））和一个奇偶校验位（\（p_j \\ ））表明不等号的方向：


$$ h_j(x) = \begin{cases}
 & 1 , if p_j f_j < p_j \Theta_j \\
 & 0 , otherwise
\end{cases} $$

Herexis a 24x24 pixel sub-window of an image。In practice no single feature can perform the classification task with low error. Features which are selected early in the process yield error rates between 0.1 and 0.3. Features selected in later rounds, as the task becomes more difficult, yield error rates between 0.4 and 0.5. Table 1 shows the learning algorithm.<br>
这里是图像的24x24像素子窗口。在实践中，没有单个特征可以以低误差执行分类任务。在过程早期选择的特征产出误差率在0.1和0.3之间。随着任务变得更加困难，在后面几轮中选择的特征产生0.4到0.5之间的错误率。表1显示了学习算法。
